__Вопросы для повторения:__
* ...
* ...
* ...

<br />

### Лекция 12. Undefined behavior

https://en.cppreference.com/w/cpp/language/ub

http://blog.llvm.org/2011/05/what-every-c-programmer-should-know.html

http://blog.llvm.org/2011/05/what-every-c-programmer-should-know_14.html

http://blog.llvm.org/2011/05/what-every-c-programmer-should-know_21.html

<br />

##### ill-formed / well-formed program

* __ill-formed__ программа - содержит синтаксические или диагностируемые семантические ошибки. Компилятор должен выдать на такие программу сообщение (error или warning).


* __ill-formed no diagnostic required__ программа - содержит недиагностируемые семантические ошибки (либо их вычислительно дорого диагностировать). Пример: ODR violation нельзя определить во время компиляции.


__Важный момент__: на уровне стандарта закреплено, что компилятор имеет false-positive ошибки при ответе на вопрос "это программа?".

Т.е. стандарт в некоторых случаях разрешает компилятору компилировать тексты, не являющиеся программами с точки зрения стандарта.

(Потому что не все ошибки диагностируемы)  

<br />

##### Разлные степени ответственности компиляторов

* __implementation-defined behavior__ - разрешается различное поведение стандартом. Конкретное поведение строго документируется реализацией (компилятор + его опции, архитектура, стандартная библиотека), и в одних реализациях всегда одинакого. Примеры:
    * тип `std::size_t`
    * размер `int`
    * какую строку возвращает `std::bad_alloc::what`
    
    
* __unspecified behavior__ - разрешается различное поведение стандартом. Поведение может отличаться даже в рамках одной и той же реализации. Классический пример:
    * порядок вычисления аргументов функции (при повторном вызове порядок уже может быть иной)


* __undefined behavior__ - компилятор / программа не несёт никакой ответственности за то, что будет происходить дальше. Компиляторы не обязаны диагностировать undefined behavior. Примеры:
    * доступ за границы массива
    * арифметика указателей за границами массива
    * операции над указателем на удалённый объект
    * переполнение знаковых целых
    * разыменование нулевого указателя
    * доступ до объекта через указатель другого типа
    * использование неинициализированной переменной (`int x; std::cout << x;`)
    * побитовый сдвиг больше чем размер типа (`uint32_t x = 1; x << 32;`). Почему так:
        * X86 truncates 32-bit shift amount to 5 bits (so a shift by 32-bits is the same as a shift by 0-bits)
        * but PowerPC truncates 32-bit shift amounts to 6 bits (so a shift by 32 produces zero)
    * ...

Древняя пасхалка в gcc: если компилятор встречал определённого типа конструкцию в коде (то ли undefined, то ли implementation defined), то он запускал игрушку

https://feross.org/gcc-ownage/

<br />

* implementation defined - иди читай документацию
* unspecified - иди читай код, убеждайся, что всё нормально
* undefined - срочно править

<br />

__Вопросы для обсуждения__:

* зачем нужен undefined behavior?

<details>
<summary>Подсказка</summary>
<p>

1. иногда без него нельзя. Пример: ODR во время компиляции.
2. для скорости программы. Пример: `vector::operator[]` - без проверок на выход за границы
3. для генерации более быстрого кода компилятором. Компилятор делает вид, что программист умный, "ub"-сценариев быть не может, и на базе этого применяет некоторые оптимизации. Примеры - далее.

Ответственность за проверку корректности перекладывается с инструмента на программиста

</p>
</details>

* Основная критика UB:

    * Ответственность на человеке
    * Люди... далеки от идеала
    

* Почему критика оправдана?

<details>
<summary>Подсказка</summary>
<p>

     * потому что работает принцип Парето: 10% кода съедают 90% времени
     * "выжимать" performance нужно только в 10% кода
     * в остальных 90% можно иметь медленный код без граблей
     * проблемы undefined behavior распространяются на 100% кода
     
Замечание: Rust и его unsafe-секции - попытка реализовать этот баланс. В 10% кода пишем unsafe-секции, где страх и ужас, в 90% кода пишем безопасный код.

</p>
</details>

<br />

##### Пример как UB помогает компилятору генерировать более эффективный код

Нарушение __Type Rules__ - undefined behavior. Type Rules (за хитрыми исключениями типа `char/void/union`) запрещает обращаться к объектам одного типа через другой тип.

Т.е. запрещено взять массив `float`-ов как `float*`, перекастить его через `int*` и дальше работать как с `int`.

Это даёт компилятору такой код:

```c++
float *P;
void zero_array() {
    for (int i = 0; i < 10000; ++i)
        p[i] = 0.0f;
}
```

Оптимизировать в такой (в разы быстрее):

```c++
float *p;
void zero_array() {
    memset(p, 0, 40'000);
}
```

У компилятора есть гарантии через ub, что в выражении `p[i] = 0.f;` не будет перезаписан сам `p`, т.к. типы `float` и `float*` разные.

Если какой-то программист решить нарушить Type Tules:

```c++
p = (float*)&p;
zero_array();
```

то изначальный и оптимизированный варианты неэквивалентны, но понятие UB позволяет компилятору с честными голубыми глазами применять оптимизации, программист сам виноват.

<br />

##### В случае UB разный порядок оптимизаций может дать разный результат

Рассмотрим пример (упрощённая версия одного бага из linux kernel):
    
```c++
void f(int *p) {
    int dead = *p;  // unused variable
    if (p == 0)     // check against nullptr after derefence
        return;
    *p = 4;
}
```

Рассмотрим 2 оптимизации компилятора:
* Dead Code Elimination
* Redundant Null Check Elimination

__Вариант 1__:
        
* шаг 1: применили dead code elimination

```c++
void f(int *p) {
    // int dead = *p;  // deleted by the optimizer.
    if (p == 0)
        return;
    *p = 4;
}
```

* шаг 2: применили redundant null check elimination

```c++
void f(int *p) {
    if (p == 0)   // Null check is not redundant, and is kept.
        return;
    *p = 4;
}
```

__Вариант 2:__

* шаг 1: применили redundant null check elimination

```c++
void f(int *p) {
    int dead = *p;
    if (false)  // p was dereferenced by this point, so it can't be null 
        return;
    *p = 4;
}
```

* шаг 2: применили dead code elimination

```c++
void f(int *p) {
    *p = 4;
}
```

<br />

##### Пример про удаление содержимого диска

```c++
static void (*FP)() = 0;

static void impl() {
    system("rm -rf /");
}

void set() {
  FP = impl;
}

void call() {
    FP();
}
```

Т.к. использование непроинициализированной переменной - UB, по-другому `FP` проинициализировать нельзя, то clang оптимизирует такой код в:

```c++
void set() {}

void call() {
    system("rm -rf /");
}
```

<br />

##### Примеры, в что может компилироваться ub

Объяснить каждый пример, почему компилятор генерит такой выхлоп

Пример:


```c++
int foo(int x) {
    return x + 1 > x; // int overflow
}
```

`-->` (gcc 9.2)

```asm
# always returns true
foo(int):
        movl    $1, %eax
        ret
```

<br />

Пример:

```c++
int f(int x, int y)
{
    if (x > 500000 && y > 500000)
        return x * y;
    return 42;
}
```

`-->` (gcc 9.2)

```c++
f(int, int):
        cmp     edi, 500000
        jle     .L3
        cmp     esi, 500000
        jle     .L3
        mov     eax, 2147483647
        ret
.L3:
        mov     eax, 42
        ret
```

<br />

Пример:

```c++
int table[4] = {};
bool exists_in_table(int v)
{
    for (int i = 0; i <= 4; i++)  // out-of-boundary access
        if (table[i] == v)
            return true;
    return false;
}
```

`-->`

```asm
# always returns true
exists_in_table(int):
        movl    $1, %eax
        ret
```

<br />

Пример:

```c++
std::size_t f(int x)
{
    std::size_t a;  // uninitialized
    if (x)  // either x nonzero or UB
        a = 42;
    return a;  // might be an access to uninitialized
}
```

`-->`

```asm
# always returns 42
f(int):
        mov     eax, 42
        ret
```

<br />

Пример:

```c++
bool p;

// access to uninitialized
if (p)
    std::puts("p is true");

// access to uninitialized
if (!p)
    std::puts("p is false");
```

output:

```
p is true
p is false
```

<br />

Пример - опровержение теоремы Ферма:
    
```c++
#include <iostream>
 
int fermat() {
  const int MAX = 1000;
  int a = 1, b = 1, c = 1;
  // Endless loop with no side effects is UB
  while (1)
  {
    if (a*a*a == b*b*b + с*c*c)
        return 1;
    a++;
    if (a > MAX) { a = 1; b++; }
    if (b > MAX) { b = 1; c++; }
    if (c > MAX) { c = 1; }
  }
  return 0;
}
 
int main() {
  if (fermat())
    std::cout << "Fermat's Last Theorem has been disproved.\n";
  else
    std::cout << "Fermat's Last Theorem has not been disproved.\n";
}
```

`-->`

```
Fermat's Last Theorem has been disproved.
```

<br />

##### Как избежать проблем с UB?
* поменять язык
* статические анализаторы кода
* санитары (asan, msan, tsan, ubsan ...), valgrind, drmemory ... - динамические анализаторы кода
* различные встроенные отладочные механизмы. Примеры:
    * проверки корректности [внутри имплементаций STL](https://gcc.gnu.org/onlinedocs/libstdc++/manual/debug_mode.html)
    * проверки кучи на целостность после операций с аллокаторами в windows [CrtSetDbgFlag](https://docs.microsoft.com/ru-ru/cpp/c-runtime-library/reference/crtsetdbgflag?view=vs-2019)
    * windows-specific [debugging tools](https://docs.microsoft.com/en-us/windows-hardware/drivers/debugger/gflags)
* повышать свой уровень грамотности С++

<br />

__Резюме:__
* implementation defined - иди читай документацию
* unspecified - иди читай код, убеждайся, что всё нормально
* undefined - срочно править
* регулярно гонять статические анализаторы по коду (и читать их отчёты)
* регулярно гонять тесты и основные сценарии под санитарами (и читать их отчёты)
* `constexpr` to the rescue! (как `constexpr` ловит ub - поговорим в продолжении курса)